In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

#for statement check            
#df.head
#print(max(df['id_path']))

In [ ]:
#table 3 = pr
#(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)
df_reg_exp = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)

id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True, drop_absorbed = True)  
fe_res = mod.fit()  
print(fe_res) 
#value chekced